## Tp2 ift 870
Quentin Levieux<br>
Frederic Giasson

In [338]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import numbers

In [339]:
package = pd.read_csv("Package2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
display("Dimension de package.csv : ",package.shape)
product = pd.read_csv("Product2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
print("Dimension de product.csv: ",product.shape)
print("Nombre d'attributs de package:", len(package.columns))
print("Nombre d'attributs de product:", len(product.columns))
print("\nColonne de package: \n",package.columns.values)
print("\nColonne de product: \n",product.columns.values)
commonAttributs = list(set(package).intersection(product))
print("\nAttributs commun au deux tables :\n",commonAttributs)

'Dimension de package.csv : '

(173887, 8)

Dimension de product.csv:  (93238, 20)
Nombre d'attributs de package: 8
Nombre d'attributs de product: 20

Colonne de package: 
 ['PRODUCTID' 'PRODUCTNDC' 'NDCPACKAGECODE' 'PACKAGEDESCRIPTION'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'NDC_EXCLUDE_FLAG'
 'SAMPLE_PACKAGE']

Colonne de product: 
 ['PRODUCTID' 'PRODUCTNDC' 'PRODUCTTYPENAME' 'PROPRIETARYNAME'
 'PROPRIETARYNAMESUFFIX' 'NONPROPRIETARYNAME' 'DOSAGEFORMNAME' 'ROUTENAME'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'MARKETINGCATEGORYNAME'
 'APPLICATIONNUMBER' 'LABELERNAME' 'SUBSTANCENAME'
 'ACTIVE_NUMERATOR_STRENGTH' 'ACTIVE_INGRED_UNIT' 'PHARM_CLASSES'
 'DEASCHEDULE' 'NDC_EXCLUDE_FLAG' 'LISTING_RECORD_CERTIFIED_THROUGH']

Attributs commun au deux tables :
 ['STARTMARKETINGDATE', 'NDC_EXCLUDE_FLAG', 'ENDMARKETINGDATE', 'PRODUCTNDC', 'PRODUCTID']


## Types des attributs

In [340]:
print("Types des attributs pour package :\n\n",package.dtypes)
print("\nTypes des attributs pour product :\n\n",product.dtypes)

Types des attributs pour package :

 PRODUCTID              object
PRODUCTNDC             object
NDCPACKAGECODE         object
PACKAGEDESCRIPTION     object
STARTMARKETINGDATE      int64
ENDMARKETINGDATE      float64
NDC_EXCLUDE_FLAG       object
SAMPLE_PACKAGE         object
dtype: object

Types des attributs pour product :

 PRODUCTID                            object
PRODUCTNDC                           object
PRODUCTTYPENAME                      object
PROPRIETARYNAME                      object
PROPRIETARYNAMESUFFIX                object
NONPROPRIETARYNAME                   object
DOSAGEFORMNAME                       object
ROUTENAME                            object
STARTMARKETINGDATE                    int64
ENDMARKETINGDATE                    float64
MARKETINGCATEGORYNAME                object
APPLICATIONNUMBER                    object
LABELERNAME                          object
SUBSTANCENAME                        object
ACTIVE_NUMERATOR_STRENGTH            object
ACTIVE_INGR

### Valeurs Manquantes

In [341]:
manquantePackage = package[package.isnull().any(axis=1)]
#display(manquantePackage)
print(manquantePackage.shape)

manquanteProduct = product[product.isnull().any(axis=1)]
#display(manquanteProduct)
print(manquanteProduct.shape)

print("Valeurs manquante par colonne dans package:\n")
for col in package.columns:
    nb = package[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans package :", len(package))

print("\nValeurs manquante par colonne dans product:\n")
for col in product.columns:
    nb = product[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans product:", len(product))

(167507, 8)
(93238, 20)
Valeurs manquante par colonne dans package:

PRODUCTID    0
dtype: int64
PRODUCTNDC    1500
dtype: int64
NDCPACKAGECODE    2346
dtype: int64
PACKAGEDESCRIPTION    0
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    167431
dtype: int64
NDC_EXCLUDE_FLAG    0
dtype: int64
SAMPLE_PACKAGE    0
dtype: int64
Nombre total de rangees dans package : 173887

Valeurs manquante par colonne dans product:

PRODUCTID    1560
dtype: int64
PRODUCTNDC    0
dtype: int64
PRODUCTTYPENAME    0
dtype: int64
PROPRIETARYNAME    6
dtype: int64
PROPRIETARYNAMESUFFIX    83075
dtype: int64
NONPROPRIETARYNAME    4
dtype: int64
DOSAGEFORMNAME    0
dtype: int64
ROUTENAME    1932
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    88915
dtype: int64
MARKETINGCATEGORYNAME    0
dtype: int64
APPLICATIONNUMBER    13097
dtype: int64
LABELERNAME    0
dtype: int64
SUBSTANCENAME    2309
dtype: int64
ACTIVE_NUMERATOR_STRENGTH    2309
dtype: int64
ACTIVE_INGRED_UNIT  

### incohérences intra-attributs

In [342]:
#Méthode pour déterminer si la date est valide
def datee(date):
    annee = int(str(date)[0:4])
    mois = int(str(date)[4:6])
    jour = int(str(date)[6:8])
    if annee<1800 or annee > 2300 or mois>12 or mois<1:
        return False
    if jour<1 or jour>31:
        return False
    if mois== 2 and jour>29:
        return False
    if mois== 4 and jour>30:
        return False
    if mois== 6 and jour>30:
        return False
    if mois== 9 and jour>30:
        return False
    if mois== 11 and jour>30:
        return False
    return True


### Table Package

In [343]:
samplePackageY = package[package['SAMPLE_PACKAGE'] != 'Y']
samplePackageN = samplePackageY[samplePackageY['SAMPLE_PACKAGE'] != 'N']
print("Nombre de rangee avec une valeur autre que Y ou N dans la colonne sample_Package : ",len(samplePackageN))

Nombre de rangee avec une valeur autre que Y ou N dans la colonne sample_Package :  0


In [344]:
dd = package[~package['NDC_EXCLUDE_FLAG'].isnull()]
ndcFlagYnotNull = dd['NDC_EXCLUDE_FLAG']
flagErr =[]
for flag in ndcFlagYnotNull:
    if str(flag) not in ['Y', 'E', 'N', 'I']:
        flagErr.append(flag)

print("Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne NDC_EXCLUDE_FLAG : ",len(flagErr))

Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne NDC_EXCLUDE_FLAG :  0


In [345]:
xx = package[~package['ENDMARKETINGDATE'].isnull()]
endMarketingDate = xx['ENDMARKETINGDATE']
erreurEndMarketingDate = []
for date in endMarketingDate:
    if not datee(date):
        erreurEndMarketingDate.append(date)
print("Nombre incoherence dans endMarketingDate: ", len(erreurEndMarketingDate))

Nombre incoherence dans endMarketingDate:  0


In [346]:
yy = package[~package['STARTMARKETINGDATE'].isnull()]
startMarketingDate = yy['STARTMARKETINGDATE']
erreurStartMarketingDate = []
for date in startMarketingDate:
    if not datee(date):
        erreurStartMarketingDate.append(date)
print("Nombre incoherence dans startMarketingDate: ", len(erreurStartMarketingDate))

Nombre incoherence dans startMarketingDate:  243


In [347]:
vv = package[~package['PRODUCTNDC'].isnull()]
ndcErrindex = []

for ndc in vv['PRODUCTNDC'].unique():
    ndca = ndc.strip('-')
    mot = str(ndc)
    if not any(char.isdigit() for char in ndca):
           ndcErrindex.append(ndc)
    if not mot.count("-")==1:#doit contenir seulement 1 tiret
        ndcErrindex.append(ndc)
print("Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres:",len(ndcErrindex))

Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres: 151


In [348]:
#NDCPACKAGECODE
fd = package[~package['NDCPACKAGECODE'].isnull()]
ndcPCKGcodeindex = []

for ndc in fd['NDCPACKAGECODE'].unique():
    ndca = ndc.strip('-')
    mot = str(ndc)
    if not any(char.isdigit() for char in ndca):
           ndcPCKGcodeindex.append(ndc)
    if not mot.count("-")==2:#doit contenir seulement 2 tirets
        ndcPCKGcodeindex.append(ndc)
print("Nombres de NDCPACKAGECODE n'ayant pas 2 tirets ou contenant des lettres:",len(ndcPCKGcodeindex))

Nombres de NDCPACKAGECODE n'ayant pas 2 tirets ou contenant des lettres: 62


In [349]:
bb = package[~package['PRODUCTID'].isnull()]
print("Nombres de differentes possibilites pour PRODUCTID:", len(bb['PRODUCTID'].unique()))
errID = bb[bb['PRODUCTID'].duplicated()== True]
errID = errID['PRODUCTID']
print("Nombres de Product pas unique:",len(errID))

fde = package[~package['PRODUCTID'].isnull()]
ndcProductIDindex = []

for ndc in fde['PRODUCTID'].unique():
    mot = str(ndc)
    if mot.count("-")==-1:#Ne contient pas de tirets
        ndcProductIDindex.append(ndc)
print("Nombres de NDCPACKAGECODE n'ayant pas de tiret:",len(ndcProductIDindex))

Nombres de differentes possibilites pour PRODUCTID: 93084
Nombres de Product pas unique: 80803
Nombres de NDCPACKAGECODE n'ayant pas de tiret: 0


En ce qui attrait au colonnes <b>PACKAGEDESCRIPTION</b> de la table PACKAGE, il est difficile de d'avoir un critère qui permet de savoir si il y a une incoherence dans les données de la colonne, le site n'offre pas plus de details. 
<br><br>Dans la colonne <b>PRODUCTID</b> nous pouvons voir qu'il y a 80803 identifiants qui ne sont pas unique, nous pouvons soupconner une incoherence d'autant plus que l'autre table n'a pas de double dans cette colonne. 

### Table product


In [350]:
#Verification de la date pour LISTING_RECORD_CERTIFIED_THROUGH
xx = product[~product['LISTING_RECORD_CERTIFIED_THROUGH'].isnull()]
listingRecordDate = xx['LISTING_RECORD_CERTIFIED_THROUGH']
erreurListingRecordDate = []
for date in listingRecordDate:
    if not datee(date):
        erreurListingRecordDate.append(date)
print("Nombre incoherence dans LISTING_RECORD_CERTIFIED_THROUGH: ", len(erreurListingRecordDate))

Nombre incoherence dans LISTING_RECORD_CERTIFIED_THROUGH:  0


In [351]:
dd = product[~product['NDC_EXCLUDE_FLAG'].isnull()]
ndcFlagYnotNull = dd['NDC_EXCLUDE_FLAG']
flagErr =[]
for flag in ndcFlagYnotNull:
    if str(flag) not in ['Y', 'E', 'N', 'I']:
        flagErr.append(flag)

print("Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne NDC_EXCLUDE_FLAG : ",len(flagErr))

Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne NDC_EXCLUDE_FLAG :  0


In [352]:
hh = product[~product['DEASCHEDULE'].isnull()] 
deaScheduleNotNull = hh['DEASCHEDULE']
deaErr =[]

for dea in deaScheduleNotNull:
    if dea not in ['CI', 'CII', 'CIII', 'CIV','CV']:
        deaErr.append(dea)
print("Nombre de DEASCHEDULE n'ayant pas de valeur valide :",len(deaErr))

Nombre de DEASCHEDULE n'ayant pas de valeur valide : 0


In [353]:
yy = product[~product['STARTMARKETINGDATE'].isnull()]
startMarketingDate = yy['STARTMARKETINGDATE']
erreurStartdMarketingDate = []
for date in startMarketingDate:
    if not datee(date):
        erreurStartMarketingDate.append(date)
print("Nombre incoherence dans startMarketingDate: ", len(erreurStartdMarketingDate))

Nombre incoherence dans startMarketingDate:  0


In [354]:
xx = product[~product['ENDMARKETINGDATE'].isnull()]
endMarketingDate = xx['ENDMARKETINGDATE']
erreurEndMarketingDate = []
for date in endMarketingDate:
    if not datee(date):
        erreurEndMarketingDate.append(date)
print("Nombre incoherence dans endMarketingDate: ", len(erreurEndMarketingDate))

Nombre incoherence dans endMarketingDate:  1


In [355]:
#PEut pt l'enlever
jj = product[~product['ACTIVE_INGRED_UNIT'].isnull()]

actIngrindex = []

for ingr in jj['ACTIVE_INGRED_UNIT'].unique():
    mot = str(ingr)
    if any(char.isdigit() for char in ingr):#Contient pas des chiffres
        #print(ingr)
        actIngrindex.append(ingr)
print("Nombres de ACTIVE_INGRED_UNIT contenant un chiffre:",len(actIngrindex))
print("Differentes possibilites pour ACTIVE_INGRED_UNIT :",len(jj['ACTIVE_INGRED_UNIT'].unique()),"\n")


Nombres de ACTIVE_INGRED_UNIT contenant un chiffre: 1754
Differentes possibilites pour ACTIVE_INGRED_UNIT : 2391 



In [356]:
kk = product[~product['ACTIVE_NUMERATOR_STRENGTH'].isnull()]
erreurActiveNumerator= [isinstance(x, numbers.Number) for x in kk]
print("Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH :",len(kk['ACTIVE_NUMERATOR_STRENGTH'].unique()))
print("Il y a ",len(erreurActiveNumerator), "identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres\n")

Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH : 8769
Il y a  20 identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres



In [357]:
vv = product[~product['PRODUCTNDC'].isnull()]
#ndcErr = []
#for ndc in vv['PRODUCTNDC'].unique():
#    ndc = ndc.strip('-')
#   if not any(char.isdigit() for char in ndc):
#           ndcErr.append(ndc)
#print("Nombres de differentes possibilites pour PRODUCTNDC:",len(ndcErr))

productNDCindex = []

for ndc in vv['PRODUCTNDC'].unique():
    mot = str(ndc)
    if not mot.count("-")==1:#Ne contient pas de tirets
        productIDindex.append(ndc)
print("Nombres de ProductNDC n'ayant pas seulement 1 tiret:",len(productNDCindex))

Nombres de ProductNDC n'ayant pas seulement 1 tiret: 0


In [358]:
bb = product[~product['PRODUCTID'].isnull()]
#print("Nombres de differentes possibilites pour PRODUCTID:", len(bb['PRODUCTID'].unique()))
#errID = bb[bb['PRODUCTID'].duplicated()== True]
#errID = errID['PRODUCTID']
#print("Nombres de Product pas unique:",len(errID))

productIDindex = []

for idd in bb['PRODUCTID'].unique():
    mot = str(idd)
    if mot.count("-")==-1:#Ne contient pas de tirets
        productIDindex.append(idd)
print("Nombres de ProductID n'ayant pas de tiret:",len(productIDindex))

Nombres de ProductID n'ayant pas de tiret: 0


In [359]:

jj = product[~product['ACTIVE_INGRED_UNIT'].isnull()]
print("Differentes possibilites pour ACTIVE_INGRED_UNIT :",len(jj['ACTIVE_INGRED_UNIT'].unique()),"\n")

#La liste des valeurs possible se trouve qqpart sur le site de la FDA
ll = product[~product['ROUTENAME'].isnull()]
print("Nombres de differentes possibilites pour ROUTENAME :",len(ll['ROUTENAME'].unique()),"\n")

#La liste des valeurs possible se trouve qqpart sur le site de la FDA
zz = product[~product['DOSAGEFORMNAME'].isnull()]
print("Nombres de differentes possibilites pour DOSAGEFORMNAME :",len(zz['DOSAGEFORMNAME'].unique()),"\n")

cc = product[~product['PROPRIETARYNAMESUFFIX'].isnull()]
nameSuffix = []
for date in cc['PROPRIETARYNAMESUFFIX'].unique():
    if any(char.isdigit() for char in date):
           nameSuffix.append(date)
print("Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX :",len(nameSuffix),"\n")


Differentes possibilites pour ACTIVE_INGRED_UNIT : 2391 

Nombres de differentes possibilites pour ROUTENAME : 180 

Nombres de differentes possibilites pour DOSAGEFORMNAME : 134 

Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX : 1455 



## A voir si on garde????

<br><br>Pour la colonne <b>PHARM_CLASSES</b>, nous ne pouvons pas évaluer les incohérences, nous n'avons pas assez d'information.
<br><br>Pour la colonne <b>ACTIVE_NUMERATOR_STRENGTH</b>, il y a des valeurs qui ne sont pas des nombres. Nous ne savons pas si ce sont des incoherences cependant, nous estimons pertinent de le souligner au cas ou cela pourrait poserait probleme plus tard.
<br><br>Pour la colonne <b>SUBSTANCENAME</b> nous ne pouvons pas savoir si les attributs qui sont dans les colonnes sont bons puisque nous n'avons pas de banque permettant de valider. Il en est de meme pour <b>APPLICATIONNUMBER LABELERNAME NONPROPRIETARYNAME PRODUCTTYPENAME PROPRIETARYNAME</b>, ce sont tous de objets, nous n'avons donc pas beaucoup de details quant a la composition des attributs.
<br><br>Pour ce qui est de <b>ROUTENAME MARKETINGCATEGORYNAME</b> nous n'avons pas trouver la liste des identifiants possible afin de confirmer que tous ceux present sont valide.


#### Conclusion 
Nous avons donc analysé toutes les colonnes afin de voir si il n'y avait pas d'erreurs ou d'incohérence dans les éléments de ces colonnes. Pour les dates nous avons vérifié qu'elles comportaient un mois entre 1 et 12 et un jour entre 1 et 31. Nous ne tenons pas compte de la variance du nombre de jours entre les mois. Lorsque la colonne comportait des objets qui par le nom nous permettait de croire que ce devrait être soi seulement composé de chiffres ou seulement composé de lettres, nous avons vérifié si c'était le cas et compté les nombre d'exceptions afin de vérifier si elles étaient répandues ou pas. Force est de constaté suite à notre analyse qu'il ne semble pas y avoir d'incohérence intra-attributs.

### Incohérences inter-Attributs

Lorsque l'on considère les différentes attributs pour les tables, la seule incohérence que nous pouvons étudier est la date, s'assurer que le début n'est pas après la fin. Il se pourrait qu'un produit ait une date de fin de marketing et pas de date de début, nous analyserons toutes ces possibilitées.

In [360]:
for row in package:
    fin=package['ENDMARKETINGDATE']
    debut=package['STARTMARKETINGDATE']
    if str(debut)>= str(fin):
        print(debut,fin)
    if len(debut)==0 or len(fin)==0 :
        print("null")
        
for row in product:
    fin=product['ENDMARKETINGDATE']
    debut=product['STARTMARKETINGDATE']
    if str(debut)>= str(fin):
        print(debut,fin)
    if len(debut)==0 or len(fin)==0 :
        print("null")
        
aa = product[~product['PRODUCTNDC'].isnull()]
ss = product[~product['LABELERNAME'].isnull()]
print("Nombre de PRODUCTNDC et nombre de PRODUCTNDC unique:",len(aa), len(aa['PRODUCTNDC'].unique()))
print("Nombre de labelername et nombre de labelerName unique:",len(ss), len(ss['LABELERNAME'].unique()))

dd = product[~product['PROPRIETARYNAME'].isnull()]
ff = dd[~dd['PROPRIETARYNAME'].isna()]
print(len(ff))#['PROPRIETARYNAMESUFFIX'])

Nombre de PRODUCTNDC et nombre de PRODUCTNDC unique: 93238 91468
Nombre de labelername et nombre de labelerName unique: 93238 6611
93232


Il ne semble pas y avoir d'incoherences en ce qui concerne les dates dans les deux tables.
<br><br>Il y a une incoherence entre <b>ProductNDC</b> et <b>LabelerName</b>, il devrait y avoir le meme nombre d'attribut unique, or nous voyons qu'il y a une grande difference entre les deux.

In [361]:
display(package.head())
display(product.head())

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
